In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('/content/sample_data/datos_v2.csv', delimiter=';')

In [3]:
df.rename(columns={'Temperatura de módulo | Sensor Card / Box (1)': 'temperaturaModulo'}, inplace=True)
df.rename(columns={'Temperatura ambiente | Sensor Card / Box (1)': 'temperaturaAmbiente'}, inplace=True)
df.rename(columns={'Irradiación | Sensor Card / Box (1)': 'irradiacion'}, inplace=True)
df.rename(columns={'Fecha y hora': 'fecha'}, inplace=True)

# Quedarme con dos columnas de entrada y una de salida(irradiacion)
df = df[['temperaturaModulo','temperaturaAmbiente','irradiacion','fecha']]




In [4]:
# Eliminar los valores nulos
df.dropna(inplace=True)

In [5]:
df.head()

,temperaturaModulo,temperaturaAmbiente,irradiacion,fecha
0,"31,00","24,00","288,00",01.03.2024 15:45
1,"34,00","24,00","293,62",01.03.2024 15:50
2,"34,00","24,00","340,00",01.03.2024 15:55
3,"36,00","24,00","469,00",01.03.2024 16:00
4,"36,00","24,00","385,00",01.03.2024 16:05


In [6]:
df['fecha'] = pd.to_datetime(df['fecha'], format='%d.%m.%Y %H:%M')
df['hora'] = [i.hour for i in df['fecha']]
df['minuto'] = [i.minute for i in df['fecha']]

In [7]:
# Parsear las columnas temperatura e irradiación a float
df['irradiacion'] = df['irradiacion'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['temperaturaModulo'] = df['temperaturaModulo'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['temperaturaAmbiente'] = df['temperaturaAmbiente'].apply(lambda x: float(str(x).strip().replace(',','.')))

In [8]:
df.head()

,temperaturaModulo,temperaturaAmbiente,irradiacion,fecha,hora,minuto
0,31.0,24.0,288.00,2024-03-01 15:45:00,15,45
1,34.0,24.0,293.62,2024-03-01 15:50:00,15,50
2,34.0,24.0,340.00,2024-03-01 15:55:00,15,55
3,36.0,24.0,469.00,2024-03-01 16:00:00,16,0
4,36.0,24.0,385.00,2024-03-01 16:05:00,16,5


In [9]:
df['tiempo'] = df.apply(lambda row: row['hora'] + (row['minuto'] // 5) * 0.083, axis=1)


In [10]:
print(df.head())

   temperaturaModulo  temperaturaAmbiente  irradiacion               fecha  \
0               31.0                 24.0       288.00 2024-03-01 15:45:00   
1               34.0                 24.0       293.62 2024-03-01 15:50:00   
2               34.0                 24.0       340.00 2024-03-01 15:55:00   
3               36.0                 24.0       469.00 2024-03-01 16:00:00   
4               36.0                 24.0       385.00 2024-03-01 16:05:00   

   hora  minuto  tiempo  
0    15      45  15.747  
1    15      50  15.830  
2    15      55  15.913  
3    16       0  16.000  
4    16       5  16.083  


In [11]:
df_nuevo = df[(df['hora'] >= 6) & (df['hora'] <= 18)]


In [12]:
df_nuevo = df_nuevo[['temperaturaModulo', 'temperaturaAmbiente', 'irradiacion', 'fecha', 'hora', 'minuto', 'tiempo']]


In [13]:
# Mostrar el nuevo DataFrame
print(df_nuevo)

       temperaturaModulo  temperaturaAmbiente  irradiacion  \
0                  31.00                 24.0       288.00   
1                  34.00                 24.0       293.62   
2                  34.00                 24.0       340.00   
3                  36.00                 24.0       469.00   
4                  36.00                 24.0       385.00   
...                  ...                  ...          ...   
12395              33.00                 23.0       302.00   
12396              34.00                 23.0       303.00   
12397              32.99                 23.0       264.03   
12398              32.00                 23.0       268.00   
12399              32.00                 23.0       311.00   

                    fecha  hora  minuto  tiempo  
0     2024-03-01 15:45:00    15      45  15.747  
1     2024-03-01 15:50:00    15      50  15.830  
2     2024-03-01 15:55:00    15      55  15.913  
3     2024-03-01 16:00:00    16       0  16.000  
4    

In [14]:
# Definir las variables independientes y dependientes para ambos modelos
X1 = df_nuevo[['temperaturaAmbiente', 'tiempo']].values
y1 = df_nuevo['temperaturaModulo'].values

X2 = df_nuevo[['temperaturaModulo', 'temperaturaAmbiente', 'tiempo']].values
y2 = df_nuevo['irradiacion'].values

# Dividir los datos en conjuntos de entrenamiento y prueba para ambos modelos
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Crear y entrenar ambos modelos de regresión lineal
model2 = LinearRegression()
model2.fit(X1_train, y1_train)

model = LinearRegression()
model.fit(X2_train, y2_train)

LinearRegression()

In [15]:
y1_pred = model2.predict(X1_test)
y2_pred = model.predict(X2_test)

mse1 = mean_squared_error(y1_test, y1_pred)
r2_1 = r2_score(y1_test, y1_pred)

mse2 = mean_squared_error(y2_test, y2_pred)
r2_2 = r2_score(y2_test, y2_pred)

print(f'Modelo 2 - Error Cuadrático Medio (MSE): {mse1}, Coeficiente de Determinación (R^2): {r2_1}')
print(f'Modelo - Error Cuadrático Medio (MSE): {mse2}, Coeficiente de Determinación (R^2): {r2_2}')

Modelo 2 - Error Cuadrático Medio (MSE): 27.013073660190397, Coeficiente de Determinación (R^2): 0.8149755891258028
Modelo - Error Cuadrático Medio (MSE): 10048.688408910097, Coeficiente de Determinación (R^2): 0.8730494239052244


In [16]:
# Importar la librería numpy
import numpy as np

# Pedir al usuario que ingrese los datos
temperaturaAmbiente = float(input("Ingrese la temperatura  Ambiente (°C): "))
hora = int(input("Ingrese la hora (0-23): "))
minuto = int(input("Ingrese los minutos (0-59): "))

# Calcular el tiempo en fracciones de hora para la entrada del usuario
tiempo = hora + (minuto / 60)

# Crear el vector de características de entrada para predecir la temperatura ambiente
x_user1 = np.array([[temperaturaAmbiente, tiempo]])

# Predecir la temperatura ambiente
temperaturaModulo = model2.predict(x_user1)[0]  # Extraer el valor del array

# Crear el vector de características de entrada para predecir la irradiación
X_user = np.array([[temperaturaModulo, temperaturaAmbiente, tiempo]])

# Predecir la irradiación
irradiacion_predicha = model.predict(X_user)
print(f"La irradiación predicha es: {irradiacion_predicha[0]}")


Ingrese la temperatura  Ambiente (°C): 22
Ingrese la hora (0-23): 17
Ingrese los minutos (0-59): 10
La irradiación predicha es: 213.76227362445553
